# Test the filter time for binary files in azimuth, height, distance format

In [147]:
import os
import random
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
from contextlib import redirect_stdout
from mmdet3d.apis import LidarDet3DInferencer

In [148]:
# Initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/rachel/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/rachel/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [149]:
azimuth_resolution = 0.05
height_resolution = 0.25

In [150]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['azimuth', 'height', 'distance', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [151]:
# Returns list of dataframes of LiDAR points
def get_frames(source_dir, num_frames):
    frame_list = []
    
    # Get frame names
    files = [f for f in os.listdir(source_dir) if f.endswith('.bin')]
    # Shuffle and take desired number of files
    random.shuffle(files)
    files = files[:num_frames]
    
    # For each frame
    for file in files:
        print('.', end='')
        file_path = Path(source_dir, file)
        # Load each frame
        frame_list.append(convert_to_dataframe(file_path))
        
    return frame_list

In [152]:
# Unpickle background map dataframe and fill the empty values
def get_filter(background_map_path):
    background_distance_lookup_table = pd.read_pickle(background_map_path)
    filled_lookup_table = background_distance_lookup_table.ffill(axis=0).bfill(axis=0)
    filled_lookup_table = filled_lookup_table.ffill(axis=1).bfill(axis=1)
    return filled_lookup_table

In [153]:
def add_lookup_coords_to_ahd(points_df):
    # print(points_df)
    # Calculate the distance, azimuth, and height using vectorized operations
    azimuth, height, distance, intensity = points_df['azimuth'], points_df['height'], points_df['distance'], points_df['intensity']
    
    # Convert and scale
    azimuth_idx = (np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 100 - 18000).astype(int)
    height_idx = (np.floor((height + 30) / height_resolution) * height_resolution * 100 - 3000).astype(int)

    points_df['azimuth_idx'] = azimuth_idx
    points_df['height_idx'] = height_idx
    
    return points_df

In [154]:
def filter_frame(pre_filtered_points, lookup_table):
    # Add lookup table coordinates
    pre_filtered_grid_lookup = add_lookup_coords_to_ahd(pre_filtered_points)
    
    # Set index to ['height_idx', 'azimuth_idx']
    pre_filtered_grid_lookup_indexed = pre_filtered_grid_lookup.set_index(['height_idx', 'azimuth_idx'])
    
    # Create a Series from the lookup table with a MultiIndex
    lookup_series = lookup_table.stack()
    
    # Reindex the lookup values to align with the DataFrame's index
    lookup_values = lookup_series.reindex(pre_filtered_grid_lookup_indexed.index)
    
    # Create a mask for indices that exist in the lookup_table
    indices_in_lookup = lookup_values.index.isin(lookup_series.index)
    
    # Create the condition based on filtering criteria
    condition = (
        indices_in_lookup & (
            lookup_values.isna() | 
            (pre_filtered_grid_lookup_indexed['distance'] < lookup_values)
        )
    )
    
    # Apply the condition to filter the DataFrame
    filtered_df = pre_filtered_grid_lookup_indexed[condition].reset_index()
    
    return filtered_df

In [155]:
def convert_to_xyz(points_df):
    azimuth, height, distance, intensity = points_df['azimuth'], points_df['height'], points_df['distance'], points_df['intensity']
    
    azimuth_rad = np.deg2rad(azimuth)
    height_rad = np.deg2rad(height)
    x = distance * np.cos(height_rad) * np.sin(azimuth_rad)
    y = distance * np.cos(height_rad) * np.cos(azimuth_rad)
    z = distance * np.sin(height_rad)
    
    # Stack the computed values into a numpy array
    xyz_intensity_array = np.column_stack((x, y, z, intensity))
    
    return xyz_intensity_array

In [156]:
def time_filter(lidar_frames_list, filter_df, filter_points=True):
    # Start timer
    start = time.time()
    # For each frame
    for frame_df in lidar_frames_list:
        # Filter frame
        if filter_points:
            frame_df = filter_frame(frame_df, filter_df)
        # Convert to x, y, z
        xyz_frame = convert_to_xyz(frame_df)
        
        # Run Inferences
        inferencer(dict(points=xyz_frame))
    # Stop timer
    end = time.time()
    return end - start

In [157]:
def run_test(data_dir, background_map_path, num_frames):
    # Get dataframes
    lidar_frames_list = get_frames(data_dir, num_frames)
    filter_df = get_filter(background_map_path)
    
    total_time_filter = time_filter(lidar_frames_list, filter_df)
    
    total_time_no_filter = time_filter(lidar_frames_list, pd.DataFrame(), filter_points=False)
    
    results_filter = {
        'type': 'filtered',
        'num_frames': num_frames,
        'total_time': total_time_filter,
        'time_per_frame': total_time_filter / num_frames
    }
    
    results_no_filter = {
        'type': 'not filtered',
        'num_frames': num_frames,
        'total_time': total_time_no_filter,
        'time_per_frame': total_time_no_filter / num_frames
    }
    return [results_filter, results_no_filter]

In [158]:
data_dir = '../data/az_height_dist_points'
background_map_path = '../data/lookup_table.pkl'

In [159]:
%%capture
# Run time test and get results
time_results = run_test(data_dir, background_map_path, 500)

In [160]:
results_df = pd.DataFrame(time_results)
display(results_df)

,type,num_frames,total_time,time_per_frame
0,filtered,500,98.566750,0.197134
1,not filtered,500,43.183662,0.086367
